# Algorithm Implementation Project

In [2]:
import numpy as np
from os import getcwd

'''Loading/Preparing data'''
proj_dir = getcwd()
eval_fold = proj_dir + '\project\project\\fast\eval'

c_lines = tuple(open(eval_fold + '\c3.0', 'r'))
a_lines = tuple(open(eval_fold + '\A3.0', 'r'))

a_lines = [line.rstrip('\n') for line in a_lines]
c_lines = [line.rstrip('\n') for line in c_lines[1:]]   # First value is number of rows

size = a_lines[0].split(' ')

a_lines = [line[:-1] for line in a_lines[1:]]          # First value is number of rows/columns
a_data = [line.split(' ') for line in a_lines]

# Convert to integers
size = [int(s) for s in size]
c_data = [int(s) for s in c_lines]

for i in range(size[0]):
    for j in range(size[1]):
        a_data[i][j] = int(a_data[i][j])

In [114]:
'''Defining classes and functions to be used'''

class Data_point:
    '''
    The most basic data point, only has a nominator and a denominator
    '''
    def __init__(self, nominator, denominator):
        self.nom = nominator
        self.denom = denominator

class Data_line:
    '''
    One line of data, contains:
         - all a values as data_points
         - a boolean for the inequality sign (0 means <=, 1 means >=)
         - the c value as a data_point
    '''
    def __init__(self, a_vector, c_value, sign):
        '''Input as integers, create data_points'''
        self.sign = sign
        self.c_dp = Data_point(c_value, 1)                        # Initial denominator 1
        self.a_dps = [Data_point(a_val, 1) for a_val in a_vector] 
            
def divide(dp, dp_div):
    '''Divides data_point dp with dp_div''' 
    dp.nom *= dp_div.denom
    dp.denom *= dp_div.nom

def multiply(dp, dp_mult):
    '''Multiplies data_point dp with dp_mult'''
    dp.nom = dp.nom*dp_mult.nom
    dp.denom = dp.denom*dp_mult.denom

def add(dp, dp_add):
    '''Adds data_point dp and dp_add'''
    dp.nom = dp.nom*dp_add.denom + dp.denom*dp_add.nom
    dp.denom = dp.denom*dp_add.denom

def subtract(dp, dp_subt):
    '''Subtracts data_point dp_subt from dp'''
    dp.nom = dp.nom*dp_subt.denom - dp.denom*dp_subt.nom
    dp.denom = dp.denom*dp_subt.denom

def simplify(dp):
    '''Simplifies data_point to its lowest denominator, and fixes so only nominator can be negative'''
    if dp.denom < 0:
        dp.nom = -dp.nom
        dp.denom = -dp.denom
    
    i = 2
    while (i < dp.denom + 1):
        if (dp.denom % i == 0 and dp.nom % i == 0):
            dp.nom = dp.nom // i
            dp.denom = dp.denom // i
            i = 1
        i += 1            
        
def switch_sign(dl):
    '''Toggles between 0 <-> 1'''
    dl.sign = 1 - dl.sign

def sort_pos_neg_zero(all_data_lines, m):
    '''Sort lines by position m in the a vector: First positive, then negative, last 0's'''
    new_data_lines = []
    
    for data_line in all_data_lines:
        if data_line.a_dps[m].nom > 0:
            new_data_lines.insert(0, data_line)
        elif data_line.a_dps[m].nom < 0:
            new_data_lines.append(data_line)
        
    for data_line in all_data_lines:
        if data_line.a_dps[m].nom == 0:
            new_data_lines.append(data_line)
    
    return new_data_lines

def simplify_whole_line(dl):
    '''Calls simplify on all a data_points and the c data_point'''
    [simplify(dp) for dp in dl.a_dps]
    simplify(dl.c_dp)

def divide_whole_line(dl, dp_div):
    '''Divides whole line by data_point dp_div'''
    
    if dl.c_dp.nom != 0:
        divide(dl.c_dp, dp_div)
    for a_dp in dl.a_dps:
        if a_dp.nom != 0:
            divide(a_dp, dp_div)
    if dp_div.nom < 0: # Call simplify before?
        switch_sign(dl)
        
def get_n1n2(all_data_lines):
    '''Perform this on the sorted data to get n1, n2, and n3 values'''
    i = 0
    while (True):
        if (all_data_lines[i].sign):
            n1 = i
            break
        i += 1
    

In [122]:
dp1 = Data_point(1, 141231)
print(dp1.nom, '/', dp1.denom)
simplify(dp1)
print('Simplifies to: ', dp1.nom, '/', dp1.denom)


1 / 141231
Simplifies to:  1 / 141231


In [124]:
'''Load all data into a list'''
all_data = [Data_line(a_data[i], c_data[i], 0) for i in range(len(c_data))]

m = 2

print('Original data')
for k in range(5):
    print('row: ', k)
    print('Nominators: ', [all_data[k].a_dps[i].nom for i in range(3)])
    print('Denominators: ', [all_data[k].a_dps[i].denom for i in range(3)])
    

sorted_data = sort_pos_neg_zero(all_data, m)

# Dividing by data point m in sorted_data
for data_line in sorted_data:
    div_dp = Data_point(data_line.a_dps[m].nom, data_line.a_dps[m].denom) # Create copy of dp to divide with
    divide_whole_line(data_line, div_dp)
    simplify_whole_line(data_line)

print('\nTest: dividing whole lines by first m= ', m)

for k in range(5):
    print('row: ', k)
    print('Nominators: ' ,[sorted_data[k].a_dps[i].nom for i in range(3)])
    print('Denominators: ', [sorted_data[k].a_dps[i].denom for i in range(3)])   
        
# TODO: Keep going from here

Original data
row:  0
Nominators:  [-119, 77, -56]
Denominators:  [1, 1, 1]
row:  1
Nominators:  [11, 67, -8]
Denominators:  [1, 1, 1]
row:  2
Nominators:  [-101, -4, 75]
Denominators:  [1, 1, 1]
row:  3
Nominators:  [-5, -22, -38]
Denominators:  [1, 1, 1]
row:  4
Nominators:  [-83, -104, 28]
Denominators:  [1, 1, 1]

Test: dividing whole lines by first m= 2
row:  0
Nominators:  [-83, -26, 1]
Denominators:  [28, 7, 1]
row:  1
Nominators:  [-101, -4, 1]
Denominators:  [75, 75, 1]
row:  2
Nominators:  [17, -11, 1]
Denominators:  [8, 8, 1]
row:  3
Nominators:  [-11, -67, 1]
Denominators:  [8, 8, 1]
row:  4
Nominators:  [5, 11, 1]
Denominators:  [38, 19, 1]


In [27]:
'''Test that data points work as intended'''

Dp1 = Data_point(2,5)
Dp2 = Data_point(4,7)

divide(Dp1, Dp2)
print('\n')
print('Nom:', Dp1.nom)
print('Denom: ', Dp1.denom)

Dp1 = Data_point(2,5)

multiply(Dp1, Dp2)
print('\n')
print('Nom:', Dp1.nom)
print('Denom: ', Dp1.denom)

Dp1 = Data_point(2,5)

add(Dp1, Dp2)
print('\n')
print('Nom:', Dp1.nom)
print('Denom: ', Dp1.denom)


Dp1 = Data_point(2,5)

subtract(Dp1, Dp2)
print('\n')
print('Nom:', Dp1.nom)
print('Denom: ', Dp1.denom)

print('\nSeems to work')



Nom: 14
Denom:  20


Nom: 8
Denom:  35


Nom: 34
Denom:  35


Nom: -6
Denom:  35

Seems to work


3 10
5 1
3 50
5 1
